In [5]:
import sys
sys.path.insert(0, '/Users/ml1000278614/Documents/PythonProjects/stockTrader_NEAT')

In [6]:
from DataBase import DatabaseConfig, StockDataHandler

In [7]:
table_name = 'stock_data'
connection_str = 'sqlite:///db/neat_stock_data.db'
stock_db_config = DatabaseConfig(table_name, connection_str)
stock_db_connection = StockDataHandler(stock_db_config)

df = stock_db_connection.query_ticker('BTC-USD')
print(df)

Could not find query for ticker: BTC-USD
Attempting to download data for: BTC-USD


[*********************100%%**********************]  1 of 1 completed

Retrying query for ticker: BTC-USD
              Open          High           Low         Close       Volume  \
0      7194.892090   7254.330566   7174.944336   7200.174316  18565664997   
1      7202.551270   7212.155273   6935.270020   6985.470215  20802083465   
2      6984.428711   7413.715332   6914.996094   7344.884277  28111481032   
3      7345.375488   7427.385742   7309.514160   7410.656738  18444271275   
4      7410.451660   7544.497070   7400.535645   7411.317383  19725074095   
...            ...           ...           ...           ...          ...   
1558  68897.109375  70284.429688  68851.632812  69362.554688  21204930369   
1559  69362.554688  72715.359375  69064.242188  71631.359375  37261432669   
1560  71632.500000  71742.507812  68212.921875  69139.015625  36426900409   
1561  69140.242188  71093.429688  67503.562500  70587.882812  38318601774   
1562  70635.023438  71244.460938  69590.742188  70242.382812  33966594048   

       ticker        SMA_20        SMA_5